In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 一个图片输入频道，六个输出频道，5x5 的平方卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 拟合函数 y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
param = list(net.parameters())
print(len(param))
print([i.size() for i in param])

10
[torch.Size([6, 1, 5, 5]), torch.Size([6]), torch.Size([16, 6, 5, 5]), torch.Size([16]), torch.Size([120, 400]), torch.Size([120]), torch.Size([84, 120]), torch.Size([84]), torch.Size([10, 84]), torch.Size([10])]


In [9]:
input = torch.randn(1, 1, 32, 32)
print(input)
out = net(input)
print(out)

tensor([[[[ 0.3545, -0.2389,  0.0492,  ..., -1.1880, -0.0436, -0.9625],
          [ 0.3449,  0.7393, -0.0785,  ..., -0.8454,  0.2633,  1.7024],
          [-0.8438,  0.0745, -0.2478,  ..., -1.1442, -2.1179, -2.3540],
          ...,
          [ 0.2231,  1.4153, -0.4322,  ..., -1.3066,  1.0148, -0.3896],
          [-1.0219, -0.2569, -0.4848,  ...,  1.2974,  0.0229, -0.2648],
          [-1.5170, -0.7889,  1.6551,  ..., -1.4596,  2.0037, -0.7404]]]])
tensor([[ 0.0030, -0.0255, -0.0827, -0.0579,  0.0100, -0.0848,  0.0014,  0.1238,
         -0.0055, -0.0239]], grad_fn=<AddmmBackward0>)


In [10]:
net.zero_grad()

In [11]:
out.backward(torch.randn(1, 10))

In [12]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.9361, grad_fn=<MseLossBackward0>)


In [18]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [19]:
net.zero_grad() # 清零现存的梯度，避免当前计算的梯度和现存梯度叠加

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0151,  0.0144, -0.0034, -0.0299,  0.0071, -0.0341])


In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)    # weight = weight - learning_rate * gradient


In [20]:
import torch.optim as optim

# 创建你的优化器
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在你的学习循环中
optimizer.zero_grad()   # 清零
output = net(input)
# output 是模型的输出，target 是实际结果，将输出结果和实际结果的差进行输入，计算损失
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 更新模型内权重